In [2]:
# load R package
# 加载R包
library(Biostrings)
library(keras)
library(tensorflow)
library(keras3)
library(stringr)

In [3]:
# Feature encoding
# 特征编码
EncodingSeq <- function(sequence){
    A <- "100000000000000000000"
    C <- "010000000000000000000"
    D <- "001000000000000000000"
    E <- "000100000000000000000"
    F <- "000010000000000000000"
    G <- "000001000000000000000"
    H <- "000000100000000000000"
    I <- "000000010000000000000"
    K <- "000000001000000000000"
    L <- "000000000100000000000"
    M <- "000000000010000000000"
    N <- "000000000001000000000"
    P <- "000000000000100000000"
    Q <- "000000000000010000000"
    R <- "000000000000001000000"
    S <- "000000000000000100000"
    T <- "000000000000000010000"
    V <- "000000000000000001000"
    W <- "000000000000000000100"
    X <- "000000000000000000010"
    Y <- "000000000000000000001"
    encoded_seq <- lapply(sequence,function(x){
        x <- toupper(x)
        x <- gsub("A",A,x)
        x <- gsub("C",C,x)
        x <- gsub("D",D,x)
        x <- gsub("E",E,x)
        x <- gsub("F",F,x)
        x <- gsub("G",G,x)
        x <- gsub("H",H,x)
        x <- gsub("I",I,x)
        x <- gsub("K",K,x)
        x <- gsub("L",L,x)
        x <- gsub("M",M,x)
        x <- gsub("N",N,x)
        x <- gsub("P",P,x)
        x <- gsub("Q",Q,x)
        x <- gsub("R",R,x)
        x <- gsub("S",S,x)
        x <- gsub("T",T,x)
        x <- gsub("V",V,x)
        x <- gsub("W",W,x)
        x <- gsub("X",X,x)
        x <- gsub("Y",Y,x)
    })
    encoded_seq <- unlist(encoded_seq)
    return(encoded_seq)
}

In [4]:
convStringToMatrix <- function(encodedSeqs, seq_len=30){
  # ensure the character type of encodedSeqs
  encodedSeqs <- as.character(encodedSeqs)
  # create the feature matrix:
  x_array <- array(data = 0, dim = c(21,seq_len, length(encodedSeqs)))
  s <- 1 # sequence/instance index
  r <- 1 # row of the matrix
  c <- 1 # column of the matrix
  j <- 1 # index of character in the one-hot encoded string
  # store each character into the right place of 3D matrix
  while (s <= length(encodedSeqs)) {
    c <- 1
    while (c <= seq_len) {
      r <- 1
      while (r <= 21) {
        x_array[r,c,s] <- as.integer(substr(encodedSeqs[s], j,j))
        r <- r + 1
        j <- j + 1
      }
      c <- c + 1
    }
    s <- s + 1
    j <- 1
  }

  #change the index order of x_array to the one keras package required:
  x_array_new <- aperm(x_array,c(3,2,1))
  return(x_array_new)
}

In [5]:
# load model
# 载入模型
model <- load_model_hdf5("model/CNN_model_cv10_fold9.hdf5")

In [6]:
# read protein sequence
# 读取蛋白序列
protein <- readAAStringSet("example//Q8S8M9.fasta")

In [7]:
aas <- c("A","C","D","E","F","G","H","I","K","L","M","N","P","Q","R","S","T","V","W","Y")

In [8]:
# we padded the amino acid sequences with length < 60 to a fixed length using character “X”.
# 提取特征序列，如果序列长度小于60的话，则两端用字母“X”补齐
Seq <- NULL
for(p in names(protein)){
    pro <- toupper(as.character(protein[p]))
    aa <- unlist(str_split(pro,""))
    inx <- which(!aa %in% aas)
    if(sum(inx)>0){
        aa[inx] <- "X"
        pro <- paste0(aa,collapse = '')
    }
    Position <- which(aa=="K")
    AA <- c()
    base_len <- 30
    for(pos in Position){
        len <- str_length(pro)
        start <- pos-base_len
        end <- pos+base_len
        if(start<1){
            start_str <- paste(rep("X",abs(start)+1),collapse ='')
        }else{
            start_str <- ""
        }
        if(end>len){
            end_str <- paste(rep("X",end-len),collapse ='')
        }else{
            end_str <- ""
        }
        aa <- paste0(start_str,substr(pro,start,end),end_str)
        aa <- paste0(substr(aa,1,base_len),substr(aa,base_len+2,base_len*2+1))
        re <- data.frame(Gene=p,Position=pos,Sequence=aa)
        Seq <- rbind(Seq,re)
    }
}

In [9]:
Seq$encoded_seq <- EncodingSeq(Seq$Sequence)

In [10]:
Sequence  <- convStringToMatrix(Seq$encoded_seq,seq_len=30*2) 

In [11]:
# predict SUMOylation sites
# 预测SUMOylation位点
pred <- model %>% predict(Sequence)

In [12]:
# prediction score
# 预测分数
Seq$Prediction <- pred[,2]

In [13]:
Seq %>%
    dplyr::select(Gene,Position,Sequence,Prediction)

Gene,Position,Sequence,Prediction
<chr>,<int>,<chr>,<dbl>
AL6,28,XXXMEGITHPIPRTVEEVFSDFRGRRAGLIALTNDMVKFYQTCDPEKENLCLYGLPNETW,0.92102933
AL6,36,HPIPRTVEEVFSDFRGRRAGLIKALTNDMVFYQTCDPEKENLCLYGLPNETWEVNLPVEE,0.97306663
AL6,45,VFSDFRGRRAGLIKALTNDMVKFYQTCDPEENLCLYGLPNETWEVNLPVEEVPPELPEPA,0.78077066
AL6,88,WEVNLPVEEVPPELPEPALGINFARDGMQEDWVSLVAVHSDSWLLSVAFYFGARFGFGKN,0.92473471
AL6,117,EKDWVSLVAVHSDSWLLSVAFYFGARFGFGNERKRLFQMINELPTIFEVVSGNAKQSKDL,0.97586221
AL6,121,VSLVAVHSDSWLLSVAFYFGARFGFGKNERRLFQMINELPTIFEVVSGNAKQSKDLSVNN,0.97285879
AL6,142,RFGFGKNERKRLFQMINELPTIFEVVSGNAQSKDLSVNNNNSKSKPSGVKSRQSESLSKV,0.82060945
AL6,145,FGKNERKRLFQMINELPTIFEVVSGNAKQSDLSVNNNNSKSKPSGVKSRQSESLSKVAKM,0.97222149
AL6,155,QMINELPTIFEVVSGNAKQSKDLSVNNNNSSKPSGVKSRQSESLSKVAKMSSPPPKEEEE,0.92757601
